Hello World!
This notebook describes the effort to cleanup and cluster a Habits dataset.

de-duplicate : based on email i'd
               
Clustering the Data:
two methods - 

A) cluster the data  and see how many clusters are there: used **MeanShift method**


B) three bins : "signed up not used" aka less than 2 days, "completed not beyond 1 week" aka less than 8 days, "everyone else"

# Looking around the data set

In [ ]:
%reset

In [ ]:
# Import the required modules
import pandas as pd
import numpy as np
import scipy as sp

In [ ]:
# simple function to read in the user data file.
# the argument parse_dates takes in a list of colums, which are to be parsed as date format
user_data_raw_csv = pd.read_csv("/home/eyebell/local_bin/janacare/janCC/datasets/Habits-Data_upto-7th-May.csv",\
                            parse_dates = [-3, -2, -1])

In [ ]:
# import the pyexcel module
#import pyexcel as pe
#from pyexcel.ext import xls

# load the file
#records = pe.get_records(file_name="/home/eyebell/local_bin/janacare/datasets/Habits-Data_upto-7th-May.xls")
#len(records)
#for record in records:
    #print record

In [ ]:
# data metrics
user_data_raw_csv.shape # Rows , colums


In [ ]:
# data metrics
user_data_raw_csv.dtypes # data type of colums

In [ ]:
user_data_to_clean = user_data_raw_csv.copy()

In [ ]:
# Some basic statistical information on the data
#user_data_to_clean.describe()

# Data Clean up

In the last section of looking around, I saw that a lot of rows do not have any values or have garbage values(see first row of the table above).
This can cause errors when computing anything using the values in these rows, hence a clean up is required.

If a the coulums *last_activity* and *first_login* are empty then drop the corresponding row !

In [ ]:
# Lets check the health of the data set
user_data_to_clean.info()

As is visible from the last column (*age_on_platform*) data type, Pandas is not recognising it as date type format. 
This will make things difficult, so I delete this particular column and add a new one.
Since the data in *age_on_platform* can be recreated by doing *age_on_platform* = *last_activity* - *first_login* 

But on eyeballing I noticed some, cells of column *first_login* have greater value than corresponding cell of *last_activity*. These cells need to be swapped, since its not possible to have *first_login* > *last_activity*
Finally the columns *first_login*, *last_activity* have missing values, as evident from above table. Since this is time data, that in my opinion should not be imputed, we will drop/delete the columns.

In [ ]:
# Run a loop through the data frame and check each row for this anamoly, if found drop,
# this is being done ONLY for selected columns

import datetime

swapped_count = 0
first_login_count = 0
last_activity_count = 0
email_count = 0
userid_count = 0

for index, row in user_data_to_clean.iterrows():        
        if row.last_activity == pd.NaT or row.last_activity != row.last_activity:
            last_activity_count = last_activity_count + 1
            #print row.last_activity
            user_data_to_clean.drop(index, inplace=True)

        elif row.first_login > row.last_activity:
            user_data_to_clean.drop(index, inplace=True)
            swapped_count = swapped_count + 1

        elif row.first_login != row.first_login or row.first_login == pd.NaT:
            user_data_to_clean.drop(index, inplace=True)
            first_login_count = first_login_count + 1

        elif row.email != row.email: #or row.email == '' or row.email == ' ':
            user_data_to_clean.drop(index, inplace=True)
            email_count = email_count + 1

        elif row.user_id != row.user_id:
            user_data_to_clean.drop(index, inplace=True)
            userid_count = userid_count + 1

print "last_activity_count=%d\tswapped_count=%d\tfirst_login_count=%d\temail_count=%d\tuserid_count=%d" \
% (last_activity_count, swapped_count, first_login_count, email_count, userid_count)

In [ ]:
user_data_to_clean.shape

In [ ]:
# Create new column 'age_on_platform' which has the corresponding value in date type format
user_data_to_clean["age_on_platform"] = user_data_to_clean["last_activity"] - user_data_to_clean["first_login"]

In [ ]:
user_data_to_clean.info()

#### Validate if email i'd is correctly formatted

In [ ]:
from validate_email import validate_email

email_count_invalid = 0
for index, row in user_data_to_clean.iterrows():        
        if not validate_email(row.email):
            #print row.email
            user_data_to_clean.drop(index, inplace=True)
            email_count_invalid = email_count_invalid + 1

print "Number of email-id invalid: %d" % (email_count_invalid)


In [ ]:
# Check the result of last operation 
user_data_to_clean.info()

### Remove duplicates

In [ ]:
user_data_to_deDuplicate = user_data_to_clean.copy()

In [ ]:
user_data_deDuplicateD = user_data_to_deDuplicate.loc[~user_data_to_deDuplicate.email.str.strip().duplicated()]
len(user_data_deDuplicateD)

In [ ]:
user_data_deDuplicateD.info()

## Clustering

In [ ]:
# Now its time to convert the timedelta64 column named age_on_platform
def convert_timedelta64_to_sec(td64):
    ts = (td64 / np.timedelta64(1, 's'))
    return ts

user_data_deDuplicateD_timedelta64_converted = user_data_deDuplicateD.copy()
user_data_deDuplicateD_timedelta64_converted['age_on_platform'] = user_data_deDuplicateD_timedelta64_converted['age_on_platform'].apply(convert_timedelta64_to_sec)

In [ ]:
# Clustering using Mean shift

from sklearn.cluster import MeanShift, estimate_bandwidth

#x = [1,1,5,6,1,5,10,22,23,23,50,51,51,52,100,112,130,500,512,600,12000,12230]
x = pd.Series(user_data_deDuplicateD_timedelta64_converted['age_on_platform'])

X = np.array(zip(x,np.zeros(len(x))), dtype=np.int)
bandwidth = estimate_bandwidth(X, quantile=0.2)
ms = MeanShift(bandwidth=bandwidth, bin_seeding=True)
ms.fit(X)
labels = ms.labels_
cluster_centers = ms.cluster_centers_

labels_unique = np.unique(labels)
n_clusters_ = len(labels_unique)

for k in range(n_clusters_):
    my_members = labels == k
    print "cluster {0} : lenght = {1}".format(k, len(X[my_members, 0]))
    #print "cluster {0}: {1}".format(k, X[my_members, 0])
    cluster_sorted = sorted(X[my_members, 0])
    print "cluster {0} : Max = {2} days & Min {1} days".format(k, cluster_sorted[0]*1.15741e-5, cluster_sorted[-1]*1.15741e-5)

In [ ]:
# Clustering using Kmeans, not working
'''
y = [1,1,5,6,1,5,10,22,23,23,50,51,51,52,100,112,130,500,512,600,12000,12230]
y_float = map(float, y)
x = range(len(y))
x_float = map(float, x)

m = np.matrix([x_float, y_float]).transpose()


from scipy.cluster.vq import kmeans
kclust = kmeans(m, 5)

kclust[0][:, 0]

assigned_clusters = [abs(cluster_indices - e).argmin() for e in x]
'''

### Binning based on **age_on_platform** 
bin1 = 0 to 1 days

bin2 = 2 to 8 days

bin3 = 8 days and beyond

In [ ]:
user_data_binned = user_data_deDuplicateD.copy()

for index, row in user_data_deDuplicateD.iterrows():
    if row["age_on_platform"] < np.timedelta64(2, 'D'):
        user_data_binned.set_value(index, 'bin', 1)
        
    elif row["age_on_platform"] >= np.timedelta64(2, 'D') and row["age_on_platform"] < np.timedelta64(8, 'D'):
        user_data_binned.set_value(index, 'bin', 2)    
        
    elif row["age_on_platform"] >= np.timedelta64(8, 'D'):
        user_data_binned.set_value(index, 'bin', 3)

In [ ]:
print "Number of users with age_on_platform less than 2 days = %d" % len(user_data_binned[user_data_binned.bin == 1])

In [ ]:
print "Number of users with age_on_platform greater than or equal to 2 days and less than 8 days = %d" % len(user_data_binned[user_data_binned.bin == 2])

In [ ]:
print "Number of users with age_on_platform greater than 8 days = %d" % len(user_data_binned[user_data_binned.bin == 3])

In [ ]:
user_data_binned.info()

In [ ]:
# Save dataframe with binned values as CSV
user_data_binned.to_csv('user_data_binned.csv')